# Analysis of Disordered Protein Distribution, Isoelectric Point and Protein Density

Idea: Use the now known 3D localization and abundance of the proteins and combine it with 
* Protein Density
* Calculated scores for their aggregation proneness, or the amount of unstructured regions within them
* Isoelectric point (acidic or basic regions)


Hypothesis: There may or may not be locations in the synapse with special environments. These might be: 
* protein density should definitely show the PSD
* Aggreagation has been shown to happen during ageing, there may or may not be special locations in the synapse with especially high/low protein disorder
* Some proteins that are either acidic or basic might form a local environment as well


In [10]:
import numpy as np
import pandas as pd
from scipy.io import loadmat, savemat
from pathlib import Path
import glob
import os
import matplotlib.pyplot as plt
import re
import pickle
import copy
%matplotlib inline


## Reading in the necessary data tables

I use my copy number file to select for my proteins of interest. Otherwise I load the table with protein disorder scores and structure composition from Hanna Wildhagen (table_structure_elements.xlsx) and the data from Cyriam et al.

In [11]:
data_folder = Path('D:/Jupyter/ProteinDisorderMap/')
# protein list from copy number file
protein_list = pd.read_excel(data_folder / "THE COPY NUMBER FILE_SE0.xlsx")
protein_list = protein_list.drop([0])
protein_list = protein_list.reset_index(drop = True)
protein_list = protein_list["Gene"]

#table from Hanna Wildhagen with disorder scores and structure composition
df = pd.read_excel(data_folder / "table_structure_elements.xlsx")
df = pd.merge(protein_list, df, how = 'left', left_on = 'Gene', right_on = 'Gene_Names')
df = df.drop(columns = ['Gene_Names', 'Protein_IDs', 'Maj_Protein_IDs', 'Protein_Names', 'Fasta_Headers', 'Isoelectric point EMBOSS', 'Isoelectric point DATASELECT'])

#read in identifier dictionary to later assign the corresponding Uniprot Identifiers
id_dict = pd.read_excel(data_folder / "Protein identifier dictionary.xlsx")
id_dict = id_dict.drop(columns = ['Entry', 'Full protein names'])
id_dict['Entry name'] = id_dict['Entry name'].str.lower()
df = pd.merge(df, id_dict, how = 'left', left_on = 'Gene', right_on = 'Gene names').drop(columns = 'Gene names')

#table from Cyriam et al.
ciryam_table = pd.read_excel(data_folder / "Ciryam_supp_table.xlsx")
ciryam_table = ciryam_table.rename(columns = ciryam_table.iloc[1])
ciryam_table = ciryam_table[['Uniprot ID', 'Zagg', 'ZaggSC']]
ciryam_table = ciryam_table.drop([0, 1])
ciryam_table = ciryam_table.reset_index(drop = True)
ciryam_table = ciryam_table.dropna()
#select only proteins from human, we dont need C.elegans.
ciryam_table = ciryam_table[ciryam_table['Uniprot ID'].str.contains('human')]
df = pd.merge(df, ciryam_table, how = 'left', left_on = 'Entry name', right_on = 'Uniprot ID').drop(columns = 'Uniprot ID')

#add column with isoelectric point difference to neutral, because this makes the data interpretation easier later
df['IsoelectricPointAverageDifferenceToNeutral'] = df['IsoelectricPointAverage'] - 7
#reorder the dataframe
df = df[['Gene', 'MH identifier', 'protein name', 'Entry name', 'Length', 'Mass', 'DisorderLong', 'DisorderShort', 'IsoelectricPointAverage', 'IsoelectricPointAverageDifferenceToNeutral', 'Coil', 'ExtendedBetaSheet', 'AlphaHelix', 'StructuredRatio', 'Zagg', 'ZaggSC']]

Drop TOM20 for further analysis, because it is the only mitochondrial protein, not present in stubby and is very sparsely scattered on the mito membrane, making the plots not nice.

In [12]:
df = df.drop(df.index[df['Gene'] == 'Tomm20'])
df = df.reset_index(drop = True)

## Set up Analysis

We need the location of the lego models. The small regex is for finding the right protein name from the file to then use the correct scores from our dataframe.
The add_score function is a small wrapper to deal with situations where no score is present in the dataframe.

In [13]:
lego_path = data_folder / 'LegoModels'
spine_classes = ['Mushroom', 'Flat']
protein_search = re.compile('([^_]*)')

def add_score(score_map, copy_map, score):
    if isinstance(score, (int, float)) and ~np.isnan(score):
        score_map += copy_map * score
    return score_map

## Analysis

Loop over the spine classes and proteins (the file loop) and add the respective score to the score map.

In [25]:
for spine_class in spine_classes:
    class_path = lego_path / spine_class
    #initialize score maps
    if spine_class == 'Mushroom':
        score_maps = {score: np.zeros((41,41,9)) for score in ['Length', 'Mass', 'DisorderLong', 'DisorderShort', 'IsoelectricPointAverage', 'IsoelectricPointAverageDifferenceToNeutral', 'Coil', 'ExtendedBetaSheet', 'AlphaHelix', 'StructuredRatio', 'Zagg', 'ZaggSC']}
        copy_map_total = np.zeros((41,41,9))
    else:
        score_maps = {score: np.zeros((51,51,7)) for score in ['Length', 'Mass', 'DisorderLong', 'DisorderShort', 'IsoelectricPointAverage', 'IsoelectricPointAverageDifferenceToNeutral', 'Coil', 'ExtendedBetaSheet', 'AlphaHelix', 'StructuredRatio', 'Zagg', 'ZaggSC']}
        copy_map_total = np.zeros((51,51,7))
    
    #get list of protein files, iterate over it
    files = []
    files = glob.glob(str(class_path / '*.mat'))

    for file in files:
        copy_map = loadmat(Path(file))
        copy_map = copy_map['matrixview']
        copy_map_total += copy_map
        protein = protein_search.match(os.path.basename(file))
        scores = df.loc[df['protein name'] == protein.group(), :]
        scores = scores[['Length', 'Mass', 'DisorderLong', 'DisorderShort', 'IsoelectricPointAverage', 'IsoelectricPointAverageDifferenceToNeutral', 'Coil', 'ExtendedBetaSheet', 'AlphaHelix', 'StructuredRatio', 'Zagg', 'ZaggSC']]
        score_maps.update((score, add_score(score_map, copy_map, scores.iloc[0][score])) for score, score_map in score_maps.items())
    
    score_maps_orig = copy.deepcopy(score_maps) #Save a copy first for normalizations later on.
    
    score_maps['CopyMap'] = copy_map_total #add the map with total copy numbers to the results for the simple analysis
    
    #save the data
    with open(data_folder / (spine_class + '_scores.pkl'), 'wb') as outfile:
        pickle.dump(score_maps, outfile)
    savemat(data_folder / (spine_class + '_scores.mat'), score_maps)
    
    #Normalize data for protein concentration per voxel, to see whether the increase in aggregation prone proteins is independent of protein concentration.
    score_maps = copy.deepcopy(score_maps_orig)
    length_map = score_maps['Length']
    length_map[length_map == 0] = 1  #to prevent division by 0
    score_maps.update((score, score_map/length_map) for score, score_map in score_maps.items())
    with open(data_folder / (spine_class + '_scores_normalized_aa.pkl'), 'wb') as outfile:
        pickle.dump(score_maps, outfile)
    savemat(data_folder / (spine_class + '_scores_normalized_aa.mat'), score_maps)
    
    #Normalize data for protein copy number per voxel.
    score_maps = copy.deepcopy(score_maps_orig)
    copy_map_total[copy_map_total == 0] = 1 #to prevent division by 0.
    score_maps.update((score, score_map/copy_map_total) for score, score_map in score_maps.items())
    with open(data_folder / (spine_class + '_scores_normalized_copynum.pkl'), 'wb') as outfile:
        pickle.dump(score_maps, outfile)
    savemat(data_folder / (spine_class + '_scores_normalized_copynum.mat'), score_maps)

## Make heatmap plots

In [26]:
from matplotlib.backends.backend_pdf import PdfPages
from mpl_toolkits.axes_grid1 import ImageGrid
plt.ioff()

files = glob.glob(str(data_folder / '*.pkl'))

for file in files:
    with open(file, 'rb') as infile:
        score_maps = pickle.load(infile)
    
    with PdfPages(str(Path(file).with_suffix('')) + '.pdf') as pdf:
        for score, score_map in score_maps.items():
            fig = plt.figure()
            plt.suptitle(score)
            grid = ImageGrid(fig, 111, nrows_ncols = (3,3), share_all = True, cbar_location = 'right', cbar_mode = 'single')

            #get max and min values to scale all images the same later on
            vmin = np.min(score_map)
            vmax = np.max(score_map)

            for i, ax in enumerate(grid):
                try:
                    im = ax.imshow(score_map[:,:,i], cmap = 'inferno', vmin = vmin, vmax = vmax)
                except:
                    break
            
            ax.cax.colorbar(im)
            ax.cax.toggle_label(True)            
            pdf.savefig()
            plt.close()
        
plt.ion()

D:\Programme\anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Attempting to set identical bottom == top == 1.0 results in singular transformations; automatically expanding.
D:\Programme\anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Attempting to set identical bottom == top == 1.0 results in singular transformations; automatically expanding.


## Create plots of lego models for all proteins

To make later analysis easier, which proteins might cause the observed effects.

In [24]:
plt.ioff()
for spine_class in spine_classes:
    class_path = lego_path / spine_class
    
        #get list of protein files, iterate over it
    files = []
    files = glob.glob(str(class_path / '*.mat'))
    with PdfPages(spine_class + '_LegoModels.pdf') as pdf:
        for file in files:
            copy_map = loadmat(Path(file))
            copy_map = copy_map['matrixview']
            fig = plt.figure()
            plt.suptitle(Path(file).stem)
            grid = ImageGrid(fig, 111, nrows_ncols = (3,3), share_all = True, cbar_location = 'right', cbar_mode = 'single')
            vmin = np.min(copy_map)
            vmax = np.max(copy_map)
            for i, ax in enumerate(grid):
                try:
                    im = ax.imshow(copy_map[:,:,i], cmap = 'inferno', vmin = vmin, vmax = vmax)
                except:
                    break

            ax.cax.colorbar(im)
            ax.cax.toggle_label(True)
            pdf.savefig()
            plt.close()
            
plt.ion()

D:\Programme\anaconda3\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Attempting to set identical bottom == top == 0 results in singular transformations; automatically expanding.
D:\Programme\anaconda3\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Attempting to set identical bottom == top == 0 results in singular transformations; automatically expanding.


## Make plots of the score distributions in my dataset

To get a better feeling whether there might be groups and which proteins have especially high/low scores

In [56]:
df_onlyscores = df[['Length', 'Mass', 'DisorderLong', 'DisorderShort', 'IsoelectricPointAverage', 'IsoelectricPointAverageDifferenceToNeutral', 'Coil', 'ExtendedBetaSheet', 'AlphaHelix', 'StructuredRatio', 'Zagg', 'ZaggSC']]
#Zagg and ZaggSC are not float columns, maybe because of some weird excel formatting or whatever. I need to transform them into float.
df_onlyscores['Zagg'] = pd.to_numeric(df_onlyscores['Zagg'], errors = 'coerce')
df_onlyscores['ZaggSC'] = pd.to_numeric(df_onlyscores['ZaggSC'], errors = 'coerce')

plt.ioff()
with PdfPages('score distributions.pdf') as pdf:
    for col in df_onlyscores.columns:
        fig = plt.hist(df_onlyscores[col])
        plt.suptitle(col)
        pdf.savefig()
        plt.close()
        
plt.ion()

D:\Programme\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Programme\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
